# Kernel visualization


### Define basic variables first:
* Model parameters
* Directory variables
* Modules to be loaded

The working directory will be the TPCwithDNN repository in order to load necessary modules. Therefore, the paths in which the figures should be stored need to be absolute paths.

In [ ]:
## directory variables
# source_dir = "/lustre/alice/users/hellbaer/TPCwithDNN/tpcwithdnn/"
source_dir = "/home/maja/CERN_part/CERN/TPCML/TPCwithDNN/tpcwithdnn/"
plot_dir = "/lustre/alice/users/hellbaer/NOTESData/JIRA/ATO-439/SC-33-33-180/plots/"
model_dir = source_dir + "model_repository/v0/"
input_dir = "/lustre/alice/users/mkleiner/NOTESData/JIRA/ATO-439/nobias/SC-33-33-180/"
import os
os.chdir(source_dir)

## import modules
import numpy as np
import pandas as pd
import math
from keras.models import model_from_json
from symmetry_padding_3d import SymmetryPadding3d
from analyser import getUnitChargeResponse, getLineChargeResponse, getPointChargeResponse
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from bokeh.io import output_notebook
output_notebook()

## model parameters
nphi = 180
nr = 33
nz = 33
filters = 4
pooling = 0
dropout = 0.0
depth = 4
batch_normalization = 0
use_scaler = 0
use_sc_mean = 1
use_sc_fluc = 1
pred_dr = 1
pred_drphi = 0
pred_dz = 0
output_pred = [pred_dr, pred_drphi, pred_dz]

## print outs
print("Working directory: %s" % source_dir)
print("Model directory: %s" % model_dir)
print("Input directory: %s" % input_dir)
print("Storage directory: %s" % plot_dir)


## global plot parameters
tooltips = [("r", "(@r)"), ("phi", "(@phi)"), ("sec", "(@sector)"), ("z", "(@z)"), ("nEvTrain", "(@nEvTrain)")]
figureLayout = [
    [0, {'plot_height': 150}],
    {'sizing_mode':'scale_width'}
    ]
widgetLayout = [[0,1,2], {'sizing_mode':'scale_width'}]

<br>
<br>
<br>

## Unit charge
### Evaluation of the response of the trained models to a uniform SC density in the whole TPC as a function of the position and the number of training events

#### Unit charge: create the data
The SC density fluctuation and mean SC density can be chosen arbitrarily. The average mean SC density is ~0.4 C/cm^3. 
Example: Mean SC density = 0.4, SC density fluctuation = 0 should result in zero distortion fluctuations. 

In [ ]:
## sc density fluctuation and mean values for unit charge
unit_sc_fluc = 0.0
unit_sc_mean = 0.4

df_unit = pd.DataFrame()
for nevents in [1000, 5000, 10000, 20000]:
    ## define model suffix
    model_files = model_dir + "modelphi%d_r%d_z%d_filter%d_poo%d_drop%.2f_depth%d_batch%d_scaler%d_useSCMean%d_useSCFluc%d_pred_doR%d_dophi%d_doz%d_Nev%d" % \
    (nphi, nr, nz, \
    filters, pooling, dropout, depth, batch_normalization, use_scaler, \
    use_sc_mean, use_sc_fluc, \
    pred_dr, pred_drphi, pred_dz, \
    nevents)
    print("Model: %s" % model_files)

    dfmodel_unit = getUnitChargeResponse(nphi, nr, nz, model_files, input_dir, unit_sc_fluc, unit_sc_mean, output_pred)
    dfmodel_unit['nEvTrain'] = nevents
    df_unit = df_unit.append(dfmodel_unit, ignore_index = True)
df_unit['sector'] = df_unit['phi'] / math.pi * 9

#### Unit charge: phi dependence
* z < 5 cm

In [ ]:
## unit charge phi dependence
output_file(plot_dir + "/figResponse_unitCharge_phi.html")
figureArraySec = [
    [['sector'], ['flucDistRPred'], {"size": 4, "colorZvar": "nEvTrain"}],
    ]
widgetParamsSec = [
    ['range', ['sector']],
    ['slider', ['r']],
    ['range', ['nEvTrain']]
    ]
bokehDrawSA.fromArray(df_unit, "z<5", figureArraySec, widgetParamsSec, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayout, sizing_mode='scale_width')

#### Unit charge: 
#### r dependence
* z < 5 cm

#### z dependence
* sector 9

In [ ]:
## r dependence
output_file(plot_dir + "/figResponse_unitCharge_r.html")
figureArrayR = [
    [['r'], ['flucDistRPred'], {"size": 4, "colorZvar": "nEvTrain"}],
    ]
widgetParamsR = [
    ['slider', ['sector']],
    ['range', ['r']],
    ['range', ['nEvTrain']]
    ]
bokehDrawSA.fromArray(df_unit, "z<5", figureArrayR, widgetParamsR, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayout, sizing_mode='scale_width')

## z dependence
output_file(plot_dir + "/figResponse_unitCharge_z.html")
figureArrayZ = [
    [['z'], ['flucDistRPred'], {"size": 4, "colorZvar": "nEvTrain"}],
    ]
widgetParamsZ = [
    ['range', ['sector']],
    ['slider', ['r']],
    ['range', ['nEvTrain']]
    ]
bokehDrawSA.fromArray(df_unit, "abs(sector-9.5)<0.5", figureArrayZ, widgetParamsZ, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayout, sizing_mode='scale_width')

<br>
<br>
<br>

## Line charge
### Evaluation of the response of the trained models to a line charge with an arbitrary position and SC density. The characteristic shape of such a scenario as a function of r and phi is well-known from RUN 2 data where it is observed at several sector boundaries of IROCs. A linear z dependence is expected for a constant line charge. 

#### Example plot from RUN 2: data (left), analytical model (middle), difference (right)
* Upper panel: dr vs r, color: Δphi (distance from the line charge)
* Lower panel: drphi vs r, color: Δphi (distance from the line charge)
<img src="../figures/lineChargeRUN2_dist-vs-r.png" width=800 height=300 />

#### Line charge: create the data
The position of the line charge is defined by *line_iphi* and *line_ir* which are the bin numbers in phi and r direction. The line charge extends over the full drift (z). The sc density of the line charge is defined by *line_sc_fluc* and *line_sc_mean* which are the density fluctuation and mean density.
* Default position:
    * sector bin 90 (= sector 9.0 for nphi = 180)
    * r bin 5 (= radius 110 cm for nr = 33)
* Default charge
    * fluctuation = 0.1
    * mean = 0.4

In [ ]:
## position (iphi, ir), sc density fluctuation and mean of line charge
line_iphi = 90
line_ir = 5
line_sc_fluc = 0.1
line_sc_mean = 0.4

df_line = pd.DataFrame()
for nevents in [1000, 5000, 10000, 20000]:
    ## define model suffix
    model_files = model_dir + "modelphi%d_r%d_z%d_filter%d_poo%d_drop%.2f_depth%d_batch%d_scaler%d_useSCMean%d_useSCFluc%d_pred_doR%d_dophi%d_doz%d_Nev%d" % \
    (nphi, nr, nz, \
    filters, pooling, dropout, depth, batch_normalization, use_scaler, \
    use_sc_mean, use_sc_fluc, \
    pred_dr, pred_drphi, pred_dz, \
    nevents)
    print("Model: %s" % model_files)

    dfmodel_line = getLineChargeResponse(nphi, nr, nz, model_files, input_dir, line_iphi, line_ir, line_sc_fluc, line_sc_mean, output_pred)
    dfmodel_line['nEvTrain'] = nevents
    df_line = df_line.append(dfmodel_line, ignore_index = True)
df_line['sector'] = df_line['phi'] / math.pi * 9


#### Line charge: phi and r dependence
* z < 5 cm
* sectors 8, 9, 10
* 95 cm < r < 125 cm

In [ ]:
## line charge phi and r dependence
output_file(plot_dir + "/figResponse_lineCharge_phi-r.html")
figureLine = [
    [['sector'], ['flucSC', 'meanSC'], {"size": 4}],
    [['r'], ['flucSC', 'meanSC'], {"size": 4}],
    [['sector'], ['flucDistRPred'], {"size": 4, "colorZvar": "r"}],
    [['r'], ['flucDistRPred'], {"size": 4, "colorZvar": "sector"}],
    ]
widgetsLine = [
    ['range', ['sector']],
    ['range', ['r']],
    ['select', ['nEvTrain']]
    ]
figureLayoutLine = [
    [0, 1, {'plot_height': 250}],
    [2, 3, {'plot_height': 250}],
    {'sizing_mode':'scale_width'}
    ]
bokehDrawSA.fromArray(df_line, "z<5 & abs(sector-9)<1 & abs(r-110)<15", figureLine, widgetsLine, layout=figureLayoutLine, tooltips=tooltips, widgetLayout=widgetLayout, sizing_mode='scale_width')

#### Line charge: z dependence
* sectors 8, 9, 10
* 95 cm < r < 125 cm

In [ ]:
## line charge z dependence
output_file(plot_dir + "/figResponse_lineCharge_z.html")
figureLineZ = [
    [['z'], ['flucDistRPred'], {"size": 4, "colorZvar": "r"}],
    ]
widgetsLineZ = [
    ['range', ['sector']],
    ['range', ['r']],
    ['select', ['nEvTrain']]
    ]
bokehDrawSA.fromArray(df_line, "abs(sector-9)<1 & abs(r-110)<15", figureLineZ, widgetsLineZ, layout=figureLayout, tooltips=tooltips, widgetLayout=widgetLayout, sizing_mode='scale_width')

<br>
<br>
<br>

## Point charge
### Evaluation of the response of the trained models to a point charge with an arbitrary position and SC density. Similar scenario as for the line charge, except a characteristic z dependence is expected in addition.

#### Point charge: create the data
The position of the point charge is defined by *point_iphi*, *point_ir* and *point_iz* which are the bin numbers in phi, r and z direction. The sc density of the point charge is defined by *point_sc_fluc* and *point_sc_mean* which are the density fluctuation and mean density.
* Default position:
    * sector bin 90 (= sector 9.0 for nphi = 180)
    * r bin 5 (= radius 110 cm for nr = 33)
    * z bin 16 (= z 120 cm for nz = 33)
* Default charge
    * fluctuation = 10.0
    * mean = 0.4

In [ ]:
## position (iphi, ir, iz), sc density fluctuation and mean of point charge
point_iphi = 90
point_ir = 5
point_iz = 16
point_sc_fluc = 10.0
point_sc_mean = 0.4

df_point = pd.DataFrame()
for nevents in [1000, 5000, 10000, 20000]:
    ## define model suffix
    model_files = model_dir + "modelphi%d_r%d_z%d_filter%d_poo%d_drop%.2f_depth%d_batch%d_scaler%d_useSCMean%d_useSCFluc%d_pred_doR%d_dophi%d_doz%d_Nev%d" % \
    (nphi, nr, nz, \
    filters, pooling, dropout, depth, batch_normalization, use_scaler, \
    use_sc_mean, use_sc_fluc, \
    pred_dr, pred_drphi, pred_dz, \
    nevents)
    print("Model: %s" % model_files)

    dfmodel_point = getPointChargeResponse(nphi, nr, nz, model_files, input_dir, point_iphi, point_ir, point_iz, point_sc_fluc, point_sc_mean, output_pred)
    dfmodel_point['nEvTrain'] = nevents
    df_point = df_point.append(dfmodel_point, ignore_index = True)
df_point['sector'] = df_point['phi'] / math.pi * 9

In [ ]:
## point charge density
output_file(plot_dir + "/figResponse_pointCharge_density.html")
figurePointDensity = [
    [['sector'], ['flucSC', 'meanSC'], {"size": 4}],
    [['r'], ['flucSC', 'meanSC'], {"size": 4}],
    [['z'], ['flucSC', 'meanSC'], {"size": 4}],
    ]
figureLayoutPointDensity = [
    [0, 1, 2, {'plot_height': 250}],
    {'sizing_mode':'scale_width'}
    ]
widgetsPointDensity = [
    ['range', ['sector']],
    ['range', ['r']],
    ['range', ['z']],
    ]
bokehDrawSA.fromArray(df_point, "abs(sector-9)<1 & abs(r-110)<15 & nEvTrain==10000", figurePointDensity, widgetsPointDensity, layout=figureLayoutPointDensity, tooltips=tooltips, widgetLayout=widgetLayout, sizing_mode='scale_width')

In [ ]:
## point charge
output_file(plot_dir + "/figResponse_pointCharge_phi-r-z.html")
figurePoint = [
    [['sector'], ['flucDistRPred'], {"size": 4, "colorZvar": "r"}],
    [['r'], ['flucDistRPred'], {"size": 4, "colorZvar": "sector"}],
    [['z'], ['flucDistRPred'], {"size": 4, "colorZvar": "r"}],
    ]
figureLayoutPoint = [
    [0],
    [1],
    [2],
    {'plot_height': 150, 'sizing_mode':'scale_width'}
    ]
widgetsPoint = [
    ['range', ['sector']],
    ['range', ['r']],
    ['range', ['z']],
    ['select', ['nEvTrain']]
    ]
widgetsLayoutPoint = [[0,1],[2,3]]
bokehDrawSA.fromArray(df_point, "abs(sector-9)<1 & abs(r-110)<15", figurePoint, widgetsPoint, layout=figureLayoutPoint, tooltips=tooltips, widgetLayout=widgetsLayoutPoint, sizing_mode='scale_width')